# Jake Cipher

아핀암호와 울타리 암호를 응용한 복합 암호화 알고리즘  

=> 공백, 숫자, 특수문자는 암호화 하지 않는다.  
=> 알파벳에만 암호화를 순차적으로 적용한다.  
=> 대문자는 대문자 암호화, 소문자는 소문자로 암호화한다.

키는 두 개를 쌍으로 하여 N개를 사용한다.  
키 쌍은 곱 키와 합 키가 있어서 입력값에 해당 값을 곱한 후 합 키를 더한다.  
암호화에 사용되는 문자셋은 알파벳 26자로 연산값에 mod 26 연산을 하여 다시 문자로 치환한다.  

In [171]:
import numpy as np
import math

In [172]:
# 알파벳만 추출한다.
def alphaonly(line):
    retline=[]
    for a in line:
        if a.isalpha():
            retline.append(a)
    return retline

In [173]:
# 리스트 성분을 길이에 맞춰 출력. 
# 일정한 길이로 필드를 출력함.
# alonly: 스트링 타입인데 알파벳이 아닌 부분은 출력 안 함.
def printlist(head, al, itemlength, alonly=True):
    strlist = []
    for a in al:
        if alonly:
            if type(a)==str and (not a.isalpha()) :
                continue
        strlist.append(("{:>"+str(itemlength)+"}").format(a))
    print(head, ''.join(strlist), sep='')

In [174]:
printlist('', ['a','bb','ccc','dd'], 5)
printlist('', ['aaaa','b','cc','dddd'], 5)

    a   bb  ccc   dd
 aaaa    b   cc dddd


In [175]:
# 문자열 맵핑. 0=A, 1=B, .. 25=Z로 맵핑한다. 
amap = [chr(ord('A')+i) for i in range(26)]
printlist('index  ', list(range(26)), 3, False)
printlist('char   ', amap, 3)

index    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
char     A  B  C  D  E  F  G  H  I  J  K  L  M  N  O  P  Q  R  S  T  U  V  W  X  Y  Z


In [176]:
plaintext = 'I AM A STUDENT.'
print('plaintext=', plaintext)

plaintext= I AM A STUDENT.


In [177]:
plaincode=[ord(c)-ord('A') for c in alphaonly(plaintext)]
printlist('plain:', plaintext, 4)
printlist('code :', plaincode, 4)

plain:   I   A   M   A   S   T   U   D   E   N   T
code :   8   0  12   0  18  19  20   3   4  13  19


### 아핀 암호화

In [178]:
# 암호화
apinkeys=[(5,3), (3,1), (9,7), (5,5)]
keyindex=0
i=0
midcipher=""

plaincode=[]
oper=[]
calc1=[]
calc2=[]
for c in plaintext:
    if c.isalpha():
        code = ord(c)-ord('A')
        plaincode.append(code)
        print(i, c, code, 'key=', apinkeys[i%4])
        a = apinkeys[i%4][0]
        b = apinkeys[i%4][1]
        oper.append("*{}+{}".format(a, b))
        vv = code*a+b
        calc1.append(vv)
        enc = (vv)%26
        calc2.append(enc)
        encchar = chr(enc+ord('A'))
        print('enc=', enc, ' str=', encchar)
        midcipher+=encchar
        i+=1
    else: 
        print(c)
        midcipher+=c
print(midcipher)

0 I 8 key= (5, 3)
enc= 17  str= R
 
1 A 0 key= (3, 1)
enc= 1  str= B
2 M 12 key= (9, 7)
enc= 11  str= L
 
3 A 0 key= (5, 5)
enc= 5  str= F
 
4 S 18 key= (5, 3)
enc= 15  str= P
5 T 19 key= (3, 1)
enc= 6  str= G
6 U 20 key= (9, 7)
enc= 5  str= F
7 D 3 key= (5, 5)
enc= 20  str= U
8 E 4 key= (5, 3)
enc= 23  str= X
9 N 13 key= (3, 1)
enc= 14  str= O
10 T 19 key= (9, 7)
enc= 22  str= W
.
R BL F PGFUXOW.


In [179]:
# enc process trace
itemlength=5
printlist('plain ', plaintext, itemlength)
printlist('code  ', plaincode, itemlength)
printlist('oper  ', oper, itemlength, False)
printlist(' * +  ', calc1, itemlength)
printlist('   %  ', calc2, itemlength)
encrypted = [amap[i] for i in calc2]
printlist('affine', encrypted, itemlength)
print('encrypted (AFFINE):', midcipher)
print('size=', len(midcipher))

plain     I    A    M    A    S    T    U    D    E    N    T
code      8    0   12    0   18   19   20    3    4   13   19
oper   *5+3 *3+1 *9+7 *5+5 *5+3 *3+1 *9+7 *5+5 *5+3 *3+1 *9+7
 * +     43    1  115    5   93   58  187   20   23   40  178
   %     17    1   11    5   15    6    5   20   23   14   22
affine    R    B    L    F    P    G    F    U    X    O    W
encrypted (AFFINE): R BL F PGFUXOW.
size= 15


### 울타리암호화

In [180]:
blocksize = math.ceil(len(midcipher)/4)*4
midcipher2 = midcipher+' '*(blocksize-len(midcipher))
print('padding size=', (blocksize-len(midcipher)))
print('AFFINE encrypted block(padding):', midcipher2)
print('size=', len(midcipher2))

padding size= 1
AFFINE encrypted block(padding): R BL F PGFUXOW. 
size= 16


In [181]:
rows = 4
cols = int(blocksize/4)
print("maxtix shape ROWS=", rows, "COLS=", cols)
mat = np.asarray(list(midcipher2))
mat = mat.reshape((rows, cols))
print(mat)

maxtix shape ROWS= 4 COLS= 4
[['R' ' ' 'B' 'L']
 [' ' 'F' ' ' 'P']
 ['G' 'F' 'U' 'X']
 ['O' 'W' '.' ' ']]


In [182]:
matt = mat.T
print(matt)
matlist = list(matt.reshape(-1))
lastcipher = ''.join(matlist)

print('encrypted(ULTARI):', lastcipher)

[['R' ' ' 'G' 'O']
 [' ' 'F' 'F' 'W']
 ['B' ' ' 'U' '.']
 ['L' 'P' 'X' ' ']]
encrypted(ULTARI): R GO FFWB U.LPX 


## Decrypt

In [183]:
print("last cipher=", lastcipher, " length=", len(lastcipher))

last cipher= R GO FFWB U.LPX   length= 16


### 울타리 암호 복호화

In [184]:
blocksize = math.ceil(len(lastcipher)/4)*4
lastcipher2 = lastcipher+' '*(blocksize-len(lastcipher))

rows = 4
cols = int(blocksize/4)
print("maxtix shape ROWS=", rows, "COLS=", cols)

mat = np.asarray(list(lastcipher2))
print(mat)

maxtix shape ROWS= 4 COLS= 4
['R' ' ' 'G' 'O' ' ' 'F' 'F' 'W' 'B' ' ' 'U' '.' 'L' 'P' 'X' ' ']


In [185]:
mat = mat.reshape((cols, rows))
print(mat)
matt = mat.T
print(matt)

[['R' ' ' 'G' 'O']
 [' ' 'F' 'F' 'W']
 ['B' ' ' 'U' '.']
 ['L' 'P' 'X' ' ']]
[['R' ' ' 'B' 'L']
 [' ' 'F' ' ' 'P']
 ['G' 'F' 'U' 'X']
 ['O' 'W' '.' ' ']]


In [186]:
matlist = list(matt.reshape(-1))
middec = ''.join(matlist)
print('decrypt (ULTARI)=', middec)

decrypt (ULTARI)= R BL F PGFUXOW. 


### 아핀 암호 복호화

In [187]:
i=0
lastdec=""
midcode=[]
calc2=[]
for c in middec:
    if c.isalpha():
        code = ord(c)-ord('A')
        midcode.append(code)
        print(i, c, code, 'key=', apinkeys[i%4])
        a = apinkeys[i%4][0]
        b = apinkeys[i%4][1]
        alpha = 0
        while (alpha+code-b)%a!=0:
            alpha += 26
        dec = int ( ((alpha+code-b)/a)%26 )
        calc2.append(dec)
        print(dec)
        decchar = chr(dec+ord('A'))
        print('dec=', dec, ' str=', decchar)
        lastdec+=decchar
        i+=1
    else: 
        print(c)
        lastdec+=c
print(lastdec)

0 R 17 key= (5, 3)
8
dec= 8  str= I
 
1 B 1 key= (3, 1)
0
dec= 0  str= A
2 L 11 key= (9, 7)
12
dec= 12  str= M
 
3 F 5 key= (5, 5)
0
dec= 0  str= A
 
4 P 15 key= (5, 3)
18
dec= 18  str= S
5 G 6 key= (3, 1)
19
dec= 19  str= T
6 F 5 key= (9, 7)
20
dec= 20  str= U
7 U 20 key= (5, 5)
3
dec= 3  str= D
8 X 23 key= (5, 3)
4
dec= 4  str= E
9 O 14 key= (3, 1)
13
dec= 13  str= N
10 W 22 key= (9, 7)
19
dec= 19  str= T
.
 
I AM A STUDENT. 


In [188]:
# enc process trace
printlist('lastenc  ', lastcipher, 4)
printlist('midenc   ', midcipher, 4)
printlist('code     ', midcode, 4)
printlist('calc2    ', calc2, 4)
# printlist(' * +   ', calc1, 4)
# printlist('   %   ', calc2, 4)
decrypted = [amap[i] for i in calc2]
printlist('decrypted', decrypted, 4)


lastenc     R   G   O   F   F   W   B   U   L   P   X
midenc      R   B   L   F   P   G   F   U   X   O   W
code       17   1  11   5  15   6   5  20  23  14  22
calc2       8   0  12   0  18  19  20   3   4  13  19
decrypted   I   A   M   A   S   T   U   D   E   N   T


In [189]:
print(lastdec)

I AM A STUDENT. 


# Cipher Module

In [190]:
def encrypt(apinkeys, plain):
#     plain=plain.upper()   ## upper case only!!!

    # key valid check
    for a,b in apinkeys:
        if a%2==0:
            return ''
        if a%13==0:
            return ''

    keypair=len(apinkeys)
    i=0
    midcipher=""
    plaincode=[]
    calc1=[]
    calc2=[]
    for c in plain:
        uppercase = c.isupper()
        if c.isalpha():
            if uppercase:
                code = ord(c)-ord('A')
            else:
                code = ord(c)-ord('a')
            plaincode.append(code)
            a = apinkeys[i%keypair][0]
            b = apinkeys[i%keypair][1]
            vv = code*a+b
            calc1.append(vv)
            enc = (vv)%26
            calc2.append(enc)
            if uppercase:
                encchar = chr(enc+ord('A'))
            else:
                encchar = chr(enc+ord('a'))
            midcipher+=encchar
            i+=1
        else: 
            midcipher+=c

    blocksize = math.ceil(len(midcipher)/4)*4
    midcipher2 = midcipher+' '*(blocksize-len(midcipher))

    rows = 4
    cols = int(blocksize/4)
    mat = np.asarray(list(midcipher2))
    mat = mat.reshape((rows, cols))

    matt = mat.T
    matlist = list(matt.reshape(-1))
    lastcipher = ''.join(matlist)

    return lastcipher


In [191]:
def decrypt(apinkeys, lastcipher):
    blocksize = math.ceil(len(lastcipher)/4)*4
    lastcipher2 = lastcipher+' '*(blocksize-len(lastcipher))
    keypair=len(apinkeys)

    rows = 4
    cols = int(blocksize/4)

    mat = np.asarray(list(lastcipher2))

    mat = mat.reshape((cols, rows))
    matt = mat.T

    matlist = list(matt.reshape(-1))
    middec = ''.join(matlist)

    ### 아핀 암호 복호화

    i=0
    lastdec=""
    midcode=[]
    calc2=[]
    for c in middec:
        uppercase = c.isupper()
        if c.isalpha():
            if uppercase:
                code = ord(c)-ord('A')
            else:
                code = ord(c)-ord('a')
            midcode.append(code)
            a = apinkeys[i%keypair][0]
            b = apinkeys[i%keypair][1]
            alpha = 0
            while (alpha+code-b)%a!=0:
                alpha += 26
            dec = int ( ((alpha+code-b)/a)%26 )
            calc2.append(dec)
            if uppercase:
                decchar = chr(dec+ord('A'))
            else:
                decchar = chr(dec+ord('a'))
            lastdec+=decchar
            i+=1
        else: 
            lastdec+=c
    return lastdec    

### 모듈 시험. (암복호화 테스트)

In [192]:
# apinkeys=[(5,3), (3,1), (9,7), (5,5)]
apinkeys=[(3,3), (11,1), (15,7)]
plain='MY NAME IS JAEWOOK. I LIVE IN "SEOUL".'

lastcipher=encrypt(apinkeys, plain)
print(lastcipher)

lastdec=decrypt(apinkeys, lastcipher)
print(lastdec)

if plain.rstrip()==lastdec.rstrip():
    print("OK")
else:
    print("FAIL")

N B"FM R DSPUTXTDZONDTTQPZ " BX.B.Q R   
MY NAME IS JAEWOOK. I LIVE IN "SEOUL".  
OK


In [193]:
# 대소문자 테스트
apinkeys=[(3,3), (11,1), (15,7)]
plain='My name is JaeWook. I live in "SEOUL". I am in 5th grade.'

lastcipher=encrypt(apinkeys, plain)
print(lastcipher)

lastdec=decrypt(apinkeys, lastcipher)
print(lastdec)
if plain.rstrip()==lastdec.rstrip():
    print("OK")
else:
    print("FAIL")

Nt"ofzR  bP5u.Tgd NydBQ p "p s.cbx droBi t pM b.dxf tq  Z b 
My name is JaeWook. I live in "SEOUL". I am in 5th grade.   
OK


In [194]:
# 암복호화 테스트 
# apinkeys=[(5,3), (3,1), (9,7), (5,5)]
apinkeys=[(3,3), (11,1), (15,7)]
plain='LET\'S GET THIS PARTY STARTED!'

lastcipher=encrypt(apinkeys, plain)
print(lastcipher)

lastdec=decrypt(apinkeys, lastcipher)
print(lastdec)
if plain.rstrip()==lastdec.rstrip():
    print("OK")
else:
    print("FAIL")

KIDGT GGGCGP'IXIFB ! RR P G PYD 
LET'S GET THIS PARTY STARTED!   
OK


In [195]:
apinkeys=[(29,3), (11,1), (15,7)]
plain='LET\'S GET THIS PARTY STARTED!'

lastcipher=encrypt(apinkeys, plain)
if lastcipher=='':
    print('invalid key')
print(lastcipher)

lastdec=decrypt(apinkeys, lastcipher)
print(lastdec)
if plain.rstrip()==lastdec.rstrip():
    print("OK")
else:
    print("FAIL")

KIDGT GGGCGP'IXIFB ! RR P G PYD 
LET'S GET THIS PARTY STARTED!   
OK


### 아핀 암호의 키값에서 곱 키에 적당한 값 검사

사용되는 문자셋 개수와 서로소가 되는 값만 곱 키로 사용 가능하다.  
본 케이스의 경우는 알파벳 26을 사용하므로 26=2*13   
따라서 인수가 2, 13이 들어가는 키는 사용할 수 없다.  (2,13의 배수는 키로 사용 불가)

In [196]:
## *a+b operator test
al = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

In [197]:
# a,b = 2, 3

In [198]:
def affinecrypt(m,b):
    al = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    enccode = []
    for c in al:
        if c.isalpha():
            code = ord(c)-ord('A')
            enc1 = code*m+b
            enc2 = enc1 % 26
#             print(c, code, enc1,'=',enc2)
            enccode.append(enc2)
    enccode = np.asarray(enccode)
    print(enccode)
    enccode.sort()
    print(enccode)
    return len(list(set(enccode)))==26

In [199]:
if affinecrypt(16,2):
    print('Good (unique)')
else:
    print('Bad (duplicate)')

[ 2 18  8 24 14  4 20 10  0 16  6 22 12  2 18  8 24 14  4 20 10  0 16  6
 22 12]
[ 0  0  2  2  4  4  6  6  8  8 10 10 12 12 14 14 16 16 18 18 20 20 22 22
 24 24]
Bad (duplicate)


In [200]:
if affinecrypt(3,2):
    print('Good (unique)')
else:
    print('Bad (duplicate)')

[ 2  5  8 11 14 17 20 23  0  3  6  9 12 15 18 21 24  1  4  7 10 13 16 19
 22 25]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]
Good (unique)


In [201]:
affinecrypt(2,2)

[ 2  4  6  8 10 12 14 16 18 20 22 24  0  2  4  6  8 10 12 14 16 18 20 22
 24  0]
[ 0  0  2  2  4  4  6  6  8  8 10 10 12 12 14 14 16 16 18 18 20 20 22 22
 24 24]


False

In [202]:
affinecrypt(5,2)

[ 2  7 12 17 22  1  6 11 16 21  0  5 10 15 20 25  4  9 14 19 24  3  8 13
 18 23]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]


True

In [203]:
affinecrypt(13,2)

[ 2 15  2 15  2 15  2 15  2 15  2 15  2 15  2 15  2 15  2 15  2 15  2 15
  2 15]
[ 2  2  2  2  2  2  2  2  2  2  2  2  2 15 15 15 15 15 15 15 15 15 15 15
 15 15]


False

In [204]:
affinecrypt(14,2)

[ 2 16  4 18  6 20  8 22 10 24 12  0 14  2 16  4 18  6 20  8 22 10 24 12
  0 14]
[ 0  0  2  2  4  4  6  6  8  8 10 10 12 12 14 14 16 16 18 18 20 20 22 22
 24 24]


False

In [205]:
affinecrypt(15,2)

[ 2 17  6 21 10 25 14  3 18  7 22 11  0 15  4 19  8 23 12  1 16  5 20  9
 24 13]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]


True